# Perturbable System first example

This notebook will outline the basics of the new sire OpenMM functionality.

In [ ]:
import BioSimSpace as BSS
import sire as sr

# Merged Molecules
This section will demonstrate the creation and visualisation of perturbations using BioSimSpace and sire, the system in this case will be a simple ethane &rarr; methanol transformation

In [ ]:
ethane = BSS.Parameters.gaff("CC").getMolecule()
methanol = BSS.Parameters.gaff("CO").getMolecule()
mapping = BSS.Align.matchAtoms(ethane, methanol)
ethane = BSS.Align.rmsdAlign(ethane, methanol, mapping)
merged = BSS.Align.merge(ethane, methanol, mapping)

solvated = BSS.Solvent.tip3p(molecule=merged, box=3 * [3 * BSS.Units.Length.nanometer])

In [ ]:
# Convert BioSimSpace to sire
sire_system = sr.convert.to(solvated, "sire")

In [ ]:
for mol in sire_system.molecules():
    if mol.is_perturbable():
        temp = mol

temp.perturbation().view()

# Running simulations of perturbed systems

#### Once a perturbed molecule has been created `sire` can be used directly to run simulations and extract energy information.

Here we will run a single simulation of the above perturbation at a lambda value of 0.5
By default, lambda behaviour is controlled by a simple morph, the same as `SOMD1`

In [ ]:
import sire as sr

Perturbable sire systems can be minimised directly at any chosen lambda value, functionality here is a wrapper around openmm minimisation

In [ ]:
m = sire_system.minimisation(lambda_val=0.5)
sire_system = m.run().commit()

Production simulations can also be run using sire dynamics - this is a simple wrapper around openMM, it adds convenience such as trajectory saving & automated calculation of energies

Here, the `lambda_values` array is used to specify all lambda values at which the potential is to be calculated.

In [ ]:
d = sire_system.dynamics(lambda_value=0.5)
d.run("10ps", energy_frequency="0.1ps", lambda_windows=[0.0, 1.0])
sire_system = d.commit()
sire_system.energy_trajectory()

# OpenMM functionality

Alternatively, the perturbable sire system can be converted to openMM, resulting in a `SOMMContext`, a simple wrapper around the `OpenMM::context` class containing information on the perturbation 

In [ ]:
omm = sr.convert.to(sire_system, "openmm")
omm

Additional information regarding lambda can be set and called directly with this context

In [ ]:
omm.set_lambda(0.5)
omm.get_lambda()

Simulations can then be run directly using this context, in precisely the same manner as any other openMM context

In [ ]:
omm.getIntegrator().step(1000)

In [ ]:
omm.get_potential_energy()

In [ ]:
omm.set_lambda(0.0)
omm.get_potential_energy()